# Streamlit Dashboard Example Code

In [ ]:
import streamlit as st

st.title('The Critics Critique App')
st.header('Introduction')
st.subheader('Steps')
st.text('This is an example')
st.latex('y = x²')
st.code('if a == 1:\n    print(a)', language='python')
st.markdown("This is **text** with markdown")

0. Import libraries

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

## 1. From URL Request to DataFrame

In [2]:
def revogamers_link_retrieve():
       
    url = f'https://www.revogamers.net/analisis-w/page/1'
    html = requests.get(url).content
    soup = BeautifulSoup(html, 'lxml')
    article = soup.find('h2')

    link = article.find('a')['href']
    title = article.find('a')['title']
                
    return link, title

In [3]:
link, title = revogamers_link_retrieve()

In [4]:
print(link, title)

https://www.revogamers.net/analisis-w/analisis-de-jet-set-knights-56745/ Análisis de Jet Set Knights


In [5]:
def revogamers_streamlit_test(link, title):
    reviews_dict = {}

    review_html = requests.get(link).content
    soup = BeautifulSoup(review_html, 'lxml')

    author = soup.find('span', {'class': 'gp-post-meta gp-meta-author'}).find('a').text

    article = soup.find('div', {'class': 'gp-entry-content'}).find_all('p')
    review = [tag.text for tag in article]
    review = ' '.join(review)

    score = soup.find('div', {'class': 'gp-rating-score'}).text.strip()
    score = float(score)
    score_adj = score / 2

    return author, review, score, score_adj

In [6]:
example, review = revogamers_streamlit_test(link, title)

In [7]:
review

'Jet Set Knights nos pone en la piel de uno de los cuatro caballeros disponibles para proteger a la princesa de los monstruos que se la quieren llevar. En esencia, es un Tower Defense en el que nuestro movmiento y acciones tienen repercusión y no solo hemos de gestionar la posición de objetos.  El juego se presenta a través de escenarios de tipo arena en los que la princesa se coloca en un lugar, y hemos de defenderla como sea posible. Cada uno de los personajes que podemos manejar tiene una habilidad particular, por lo que es imortante escoger el que se adapte más a nuestro estilo. Una vez que hemos «avanzado» hasta cierto punto, el nivel finaliza, no sin antes enfrentarnos a un jefe. Los controles son sencillos y lo principal, más allá de nuestra habilidad, es entender bien los elementos del escenario y saber el orden en el que hay que derrotar enemigos para evitar que lleguen a su destino. Además, hay poderes que nos dan mejoras temporales y también hay que saber cuándo y cómo usarl

In [8]:
nlp_model = 'nlptown/bert-base-multilingual-uncased-sentiment'
tokenizer = AutoTokenizer.from_pretrained(nlp_model)
model = AutoModelForSequenceClassification.from_pretrained(nlp_model)
classifier = pipeline(
        'sentiment-analysis', 
        model=model, 
        tokenizer=tokenizer)

In [22]:
def sentiment_analysis_bert_base_multilingual_uncased(review):
    n = 1500
    points = []
    
    review_splitted = [(review[i:i + n]) for i in range(0, len(review), n)]
    global_stars = (classifier(review_splitted))

    for classification in global_stars:
        grade = int(classification['label'].split(' ')[0])
        points.append(grade)
    stars_mean = round(np.mean(points), 2)

    return stars_mean

In [23]:
stars_mean = sentiment_analysis_bert_base_multilingual_uncased(review)

In [25]:
print(stars_mean)

4.0
